In [1]:
vowels = 'аеёиоуыэюя'
consonants = 'бвгджзйклмнпрстфхцчшщъь'
alphabet = vowels + consonants

In [10]:
import copy
import numpy as np
import pandas as pd
import re
import random
from collections import defaultdict

In [80]:
def split_syllables(word):
    syllables = []
    cur_syl_tpl = {
        'syl': [],
        'has_vowel': False
    }
    cur_syl = copy.deepcopy(cur_syl_tpl)
    for i, letter in enumerate(word):
        if cur_syl['has_vowel'] is False:
            cur_syl['syl'].append(letter)
        elif letter in consonants\
             and (((i < len(word) - 1) and word[i + 1] in consonants) or (i == len(word) - 1)):
            cur_syl['syl'].append(letter)
        elif letter in ['ь', 'ъ']:
            cur_syl['syl'].append(letter)
        elif letter == "'":
            cur_syl['syl'].append(letter)
        else:
            syllables.append(''.join(cur_syl['syl']))
            cur_syl = copy.deepcopy(cur_syl_tpl)
            cur_syl['syl'].append(letter)
            
        if letter in vowels:
            cur_syl['has_vowel'] = True
    syllables.append(''.join(cur_syl['syl']))
    return syllables

In [84]:
assert '-'.join(split_syllables('вова')) == 'во-ва'
assert '-'.join(split_syllables('футбол')) == 'фут-бол'
assert '-'.join(split_syllables('стол')) == 'стол'
assert '-'.join(split_syllables('борщ')) == 'борщ'
assert '-'.join(split_syllables('эхо')) == 'э-хо'
assert '-'.join(split_syllables('баян')) == 'ба-ян'
assert '-'.join(split_syllables('транспорт')) == 'транс-порт'
assert '-'.join(split_syllables('упорядочить')) == 'у-по-ря-до-чить'
assert '-'.join(split_syllables('абракадабра')) == 'аб-ра-ка-даб-ра'
assert '-'.join(split_syllables('презумпция')) == 'пре-зумп-ци-я'
assert '-'.join(split_syllables('бельэтаж')) == 'бель-э-таж'
assert '-'.join(split_syllables('племянник')) == 'пле-мян-ник'
assert '-'.join(split_syllables('подкаблучник')) == 'под-каб-луч-ник'
assert '-'.join(split_syllables('праздник')) == 'празд-ник'
assert '-'.join(split_syllables('пресмыкающееся')) == 'прес-мы-ка-ю-ще-е-ся'
assert '-'.join(split_syllables('мерзостный')) == 'мер-зост-ный'
# assert '-'.join(split_syllables('представление'))) == 'пред-став-ле-ни-е' 
# assert '-'.join(split_syllables('русский'))) == 'русс-кий'
# assert '-'.join(split_syllables('попустительствовать'))) == 'по-пу-сти-тельст-во-вать'
# assert '-'.join(split_syllables('морская'))) == 'мор-ска-я'

In [52]:
def get_stress_pattern(word_stress):
    pattern = []
    i = len(word_stress) - 1
    while i >= 0:
        letter = word_stress[i]
        if letter in vowels:
            pattern.append("-")
            i -= 1
        elif letter == "'":
            pattern.append("'")
            i -= 2
        else:
            i -= 1
    return ''.join(reversed(pattern))


def get_stressed_syllable(word_stress):
    stressed_syllable = -1
    for letter in reversed(word_stress):
        if letter in vowels:
            stressed_syllable -= 1
        if letter == "'":
            break
    return stressed_syllable


def get_tail(word, word_stress):
    stressed_syllable = get_stressed_syllable(word_stress)
    if stressed_syllable == -1:
        syllables = split_syllables(word)
        tail = syllables[-1]
    else:
        parts = word_stress.split("'")
        tail = parts[0][-1] + parts[1]
        
    return tail

In [53]:
# https://nlpub.mipt.ru/%D0%A0%D0%B5%D1%81%D1%83%D1%80%D1%81%D1%8B
# http://www.speakrus.ru/dict/#paradigma
# http://www.speakrus.ru/dict/hagen-morph.rar
ddict = []

with open('/mnt/c/Users/admin/Downloads/hagen-morph/dict2.txt') as f:
    words, properties = [], []
    for line in f:
        line = [x.strip() for x in line.split('|')]
        if len(line) == 1:
            continue
        word, word_stress = line[0], line[2]

        if not re.match('^[А-я]+$', word)\
           or "'" not in word_stress:
            continue
            
        stress_pattern = get_stress_pattern(word_stress)
        stressed_syllable = stress_pattern.find("'") - len(stress_pattern)
        tail = get_tail(word, word_stress)
        words.append(word)
        properties.append([word_stress, stress_pattern, stressed_syllable, tail])
    ddict = dict(zip(words, properties))

In [363]:
ddict["удавка"]

["уда'вка", "-'-", -2, 'авка']

In [364]:
def find_rhyme(word):
    word_stress, stress_pattern, stressed_syllable, tail = ddict[word]
    rhymes = []
    for cur_word, properties in ddict.items():
        if properties[3] == tail:
            rhymes.append(cur_word)
    return rhymes
find_rhyme("азу")[:100]

['абзу',
 'азу',
 'бирюзу',
 'бузу',
 'ввезу',
 'везу',
 'взвезу',
 'внизу',
 'возу',
 'вползу',
 'всползу',
 'гарнизу',
 'глазу',
 'гликозидазу',
 'гомозу',
 'грозу',
 'грызу',
 'гюрзу',
 'дерезу',
 'довезу',
 'догрызу',
 'доползу',
 'егозу',
 'завезу',
 'загрызу',
 'заползу',
 'иглофрезу',
 'изгрызу',
 'каллозу',
 'канзу',
 'капилляриозу',
 'картузу',
 'кинзу',
 'кирзу',
 'кобзу',
 'козу',
 'ксендзу',
 'кузу',
 'лозу',
 'мирзу',
 'мурзу',
 'навезу',
 'нагрызу',
 'надгрызу',
 'нанизу',
 'наползу',
 'недовезу',
 'низу',
 'обвезу',
 'обгрызу',
 'обползу',
 'огрызу',
 'оползу',
 'отвезу',
 'отгрызу',
 'отползу',
 'пазу',
 'парафиляриозу',
 'перевезу',
 'перегрызу',
 'переползу',
 'пирикуляриозу',
 'повезу',
 'погрызу',
 'подвезу',
 'подгрызу',
 'подползу',
 'ползу',
 'понавезу',
 'понаползу',
 'поползу',
 'привезу',
 'приползу',
 'провезу',
 'прогрызу',
 'проползу',
 'развезу',
 'разгрызу',
 'рогозу',
 'сабзу',
 'свезу',
 'сгрызу',
 'сползу',
 'стрекозу',
 'судзу',
 'тазу',
 'тузу',
 'ув

In [423]:
def fill_pattern(phrase, rhyme_last_syllable=True):
    pattern, syllables = "", []
    for word in phrase.split():
        word = word.strip()
        pattern += get_stress_pattern(ddict[word][0])
        syllables.extend(split_syllables(ddict[word][0]))

    unfilled_pattern = pattern
    cur_pos = 0
    keys = list(ddict.keys())
    
#     np.random.seed(42)
    random_idx = np.random.choice(list(range(len(keys))), size=len(keys), replace=False)
    res = []
    for i in random_idx:
        cur_word = keys[i]
        cur_pattern = ddict[cur_word][1]
        re_pattern = "^" + cur_pattern
        cur_word_ok = False
        if re.match(re_pattern, unfilled_pattern):
            # если текущее слово короче оставшейся строки
            # и если не остался один безударный слог
            if len(cur_pattern) < len(unfilled_pattern)\
                and re.sub(re_pattern, "", unfilled_pattern) != "-":
                cur_word_ok = True
            else:
                remnance = "".join(syllables)
                parts = remnance.split("'")
                tail = parts[0][-1] + parts[1]
                if ddict[cur_word][3] == tail:
                    cur_word_ok = True

            if cur_word_ok is True:
                res.append(cur_word)
                unfilled_pattern = re.sub(re_pattern, "", unfilled_pattern)
                syllables = syllables[len(cur_pattern):]

            if len(unfilled_pattern) == 0:
                break
    return res

In [462]:
phrase = "едва открыл рот"
' '.join(fill_pattern(phrase))

'андийках филс от'

In [463]:
phrase = "внезапно оттуда со скрежетом вырвалось слово"
' '.join(fill_pattern(phrase))

'сдвои иждивенство козлищем пошляйтесь махрово'

In [454]:
ddict["эстетство"]

["эстэ'тство", "-'-", -2, 'этство']

In [54]:
def get_n_grams_dict(text):
    ngrams = defaultdict(list)
    for i in range(3, len(text)):
        ngrams[(text[i - 3], text[i - 2], text[i - 1])].append(text[i])
    
    probs = dict()
    for ngr in ngrams:
        cnt = defaultdict(int)
        for word in ngrams[ngr]:
            cnt[word] += 1
        for word in cnt:
            cnt[word] /= len(ngrams[ngr])
        probs[ngr] = dict(cnt)

    return probs        

In [55]:
# http://study.mokoron.com/
# https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
df = pd.read_csv("positive.csv", sep=';', error_bad_lines=False, header=None)

In [56]:
def cut_tweeter_special_stuff(line):
    new_line = re.sub('^RT ', '', line)
    new_line = re.sub('#.*?( |$)', '', new_line)
    new_line = re.sub('@.*?( |$)', '', new_line)
    new_line = re.sub('http.*?( |$)', '', new_line)
    return new_line
df[3] = df[3].map(cut_tweeter_special_stuff)
    

In [57]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"хоть я и школота, но поверь, у нас то же самое...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,Ну ты идиотка) я испугалась за тебя!!!,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"""Кто то в углу сидит и погибает от голода, а м...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,"Вот что значит страшилка :D\nНо блин,посмотрев...",1,0,0,0,597,16,23,1


In [60]:
text = []
for txt in df[3]:
    for word in re.findall("[\w-]+|[.,!?-];", txt):
        word = word.lower()
        if word in ddict:
            text.append(word)

In [61]:
probs = get_n_grams_dict(text)

In [37]:
def get_next_word(ngram, probs):
    p = [(k, v) for k, v in sorted(probs[ngram].items(), key=lambda item: item[1])]
    if len(p) > 1 and np.random.rand() > 0.5:
        return p[-2][0]
    else:
        return p[-1][0]

In [78]:
rnd_idx = np.random.randint(len(probs))
random_text = list(list(probs.keys())[rnd_idx])
for i in range(10):
    current_ngram = tuple(random_text[-3:])
    random_text.append(get_next_word(current_ngram, probs))
' '.join(random_text)

'легче но чувствую себя плохо по разу пересматриваю фильм пацан малым сегодня работа'